In [ ]:
#@title CV Dashboard — Cell 1 (Auth & Read)

# ===== Settings =====
# 読み取り元（元データ）
READ_GS_URL = "https://docs.google.com/spreadsheets/d/1wz20DITKF1GpIryhifiPShEbSgIVwSqlD22WEL6dpCY"  # 常に最新の月次レポート
READ_SHEET_NAME = "_ch-m"

# 書き出し先（別ブック）
WRITE_GS_URL = "https://docs.google.com/spreadsheets/d/1zw8BEwj6pxe-n7PKyC80GFr0EzIHqS3XOpuBVMVeF-4/"  # @param {type:"string"}
WRITE_SHEET_NAME = "_summary-cv" # @param {type:"string"}

# クリニック（日本語→英語）12院
CLINIC_FILTER = [
    ("池袋", "Ikebukuro"),
    ("博多", "Hakata"),
    ("仙台", "Sendai"),
    ("東京", "Tokyo"),
    ("梅田", "Umeda"),
    ("新宿", "Shinjuku"),
    ("横浜", "Yokohama"),
    ("渋谷", "Shibuya"),
    ("難波", "Namba"),
    ("札幌", "Sapporo"),
    ("天神", "Tenjin"),
    ("名古屋", "Nagoya"),
]

HALF_LIFE_MONTHS = 3   # EWMA half-life (months)
SAVE_TO_SHEET = True  # True にすると WRITE_* に書き出し

try:
    from google.colab import data_table, files as colab_files
    IN_COLAB = True
except Exception:
    data_table = None
    colab_files = None
    IN_COLAB = False

try:
    from megaton import start
except ModuleNotFoundError:
    if IN_COLAB:
        %pip install -U -q git+https://github.com/mak00s/megaton
        from megaton import start
    else:
        raise

# ===== Install & Imports =====
import sys, subprocess, importlib.util
def _pip_install(pkgs):
    try:
        missing = []
        for p in pkgs:
            mod = p.split("[")[0].replace("-", "_")
            if importlib.util.find_spec(mod) is None:
                missing.append(p)
        if missing:
            print("Installing:", missing)
            subprocess.check_call([sys.executable, "-m", "pip", "install", *missing])
    except Exception as e:
        print("⚠️ pip install failed:", e)

_pip_install(["gspread", "gspread-dataframe", "pandas", "numpy", "matplotlib"])

import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ===== Google Auth & Read =====
print("== Step 1/5: Google Auth ==")
try:
    # Colab OAuth
    try:
        from google.colab import auth as colab_auth  # type: ignore
        colab_auth.authenticate_user()
        print("✅ Colab OAuth OK")
    except Exception:
        pass

    # Default creds -> gspread
    try:
        import google.auth  # type: ignore
        from google.auth.transport.requests import Request  # type: ignore
        scopes = [
            "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive",
        ]
        creds, _ = google.auth.default(scopes=scopes)
        if hasattr(creds, "refresh") and hasattr(creds, "valid") and not creds.valid:
            creds.refresh(Request())
        gc = gspread.authorize(creds)
        print("✅ google.auth.default() OK")
    except Exception:
        # Fallbacks
        try:
            gc = gspread.oauth()
            print("✅ gspread.oauth() OK")
        except Exception:
            from google.oauth2.service_account import Credentials  # type: ignore
            import os
            sa_path = os.getenv("SERVICE_ACCOUNT_JSON")
            if not sa_path:
                raise RuntimeError("Auth failed. Use Colab OAuth or set SERVICE_ACCOUNT_JSON.")
            sa_scopes = [
                "https://www.googleapis.com/auth/spreadsheets",
                "https://www.googleapis.com/auth/drive",
            ]
            creds = Credentials.from_service_account_file(sa_path, scopes=sa_scopes)
            gc = gspread.authorize(creds)
            print("✅ Service Account OK")

    print("== Step 2/5: Read Sheet ==")
    if not READ_GS_URL or "http" not in READ_GS_URL:
        raise ValueError("Set READ_GS_URL to your Sheet URL.")
    sh_read = gc.open_by_url(READ_GS_URL)
    ws_read = sh_read.worksheet(READ_SHEET_NAME)

    # header=0（1行目にヘッダー）
    df_raw = get_as_dataframe(ws_read, evaluate_formulas=True, header=0)
    df_raw.columns = [str(c).strip() for c in df_raw.columns]
    print(f"✅ Read OK: {READ_SHEET_NAME} rows={len(df_raw)}")
    print("🧪 columns after read:", list(df_raw.columns))

    # === (UPDATE) _couns（来店カウンセリング件数） ===
    READ_SHEET_NAME_COUNS = "_couns"
    try:
        ws_couns = sh_read.worksheet(READ_SHEET_NAME_COUNS)
        # 1-basedで2行目が英語ヘッダー → header=1 (0-based)
        df_couns = get_as_dataframe(ws_couns, evaluate_formulas=True, header=1)
        df_couns.columns = [str(c).strip() for c in df_couns.columns]
        # 最小スキーマに正規化（日本語見出しだった場合もケア）
        alias = {"年月":"month", "クリニック":"clinic", "来院数":"cv"}
        df_couns = df_couns.rename(columns={k:v for k,v in alias.items() if k in df_couns.columns})
        # 不要な完全NaN行削除
        df_couns = df_couns.dropna(how="all")
        print(f"✅ Read OK: {READ_SHEET_NAME_COUNS} rows={len(df_couns)}")
        print("🧪 _couns columns:", list(df_couns.columns)[:6])
    except gspread.WorksheetNotFound:
        df_couns = None
        print("⚠️ _couns シートが見つかりません（df_couns=None）。スキップします。")

except Exception as e:
    raise SystemExit(f"❌ Auth/Read error: {e}")


In [ ]:
#@title CV Dashboard — Cell 2 (Prep, Metrics, Microcharts)

print("== Step 3/5: Preprocessing / Validation ==")

# ASCII only labels in charts
import matplotlib
matplotlib.rcParams["font.family"] = "DejaVu Sans"
matplotlib.rcParams["axes.unicode_minus"] = True

# Guard
import pandas as pd, numpy as np
if 'df_raw' not in globals() or df_raw is None:
    raise SystemExit("❌ df_raw is None. Make sure Cell 1 completed successfully.")

# クリニックの日本語→英語マップ
CLINIC_JA_TO_EN = {ja: en for ja, en in CLINIC_FILTER}
CLINIC_ALLOWED_JA = [ja for ja, en in CLINIC_FILTER]

# 前処理
_df = df_raw.copy()
_df = _df.dropna(how="all")

# 必須列
required_cols = ["month", "clinic", "channel", "medium", "source", "users", "cv"]
missing = [c for c in required_cols if c not in _df.columns]
if missing:
    raise SystemExit("❌ Required columns missing: " + str(missing))

# ★ 当月データを除外（不完全月）
#   month は 202509 のようなYYYYMM想定。タイムゾーンは日本時間ベースでOK。
current_ym = pd.Timestamp.now(tz="Asia/Tokyo").strftime("%Y%m")
_df = _df[_df["month"].astype(str) != current_ym].copy()

# 型変換
_df["month"] = pd.to_datetime(_df["month"].astype(str), format="%Y%m", errors="coerce")
if _df["month"].isna().any():
    bad = _df[_df["month"].isna()].head(5)
    raise SystemExit(f"❌ Failed to parse 'month' as YYYYMM. Top bad rows:\n{bad}")

num_cols = [c for c in ["users", "cv", "ad_cost"] if c in _df.columns]
for c in num_cols:
    _df[c] = pd.to_numeric(_df[c], errors="coerce")
if num_cols:
    _df[num_cols] = _df[num_cols].fillna(0)

# ★ チャネル条件：Direct は除外、その他は全て含める（＝Organic 限定ではない）
_df = _df[_df["channel"] != "Direct"].copy()

# ★ クリニックは12院（存在するもののみ）
_df = _df[_df["clinic"].isin(CLINIC_ALLOWED_JA)].copy()
if _df.empty:
    raise SystemExit("⚠️ No rows after clinic filter. Check CLINIC_FILTER and data.")

print("✅ Preprocess OK: rows=", len(_df))

# 集計（院×月；全チャネル合算 ※Direct除外済み）
df_m = (
    _df.groupby(["clinic", "month"], as_index=False)[["users", "cv"]]
      .sum()
      .sort_values(["clinic", "month"])
)

# 指標定義
print("== Step 4/5: Metrics (TTM / YoY_3M / EWMA) ==")
def add_metrics(group: pd.DataFrame) -> pd.DataFrame:
    g = group.sort_values("month").copy()
    g["cvr"] = np.where(g["users"] > 0, g["cv"] / g["users"], np.nan)
    g["cv_ttm12"]   = g["cv"].rolling(window=12, min_periods=1).sum()                # TTM
    g["cv_3m"]      = g["cv"].rolling(window=3,  min_periods=1).sum()
    g["cv_3m_lastyr"]= g["cv_3m"].shift(12)
    g["yoy_3m"]     = np.where(g["cv_3m_lastyr"] > 0, (g["cv_3m"]/g["cv_3m_lastyr"]) - 1, np.nan)  # 季節調整の近似
    g["cv_ewma"]    = g["cv"].ewm(halflife=HALF_LIFE_MONTHS, adjust=False).mean()    # EWMA
    return g

# groupby.apply（将来互換対応）
try:
    df_metrics = (
        df_m.sort_values(["clinic","month"])
            .groupby("clinic", group_keys=True)
            .apply(add_metrics, include_groups=False)
            .reset_index(level=0)
    )
except TypeError:
    df_metrics = (
        df_m.sort_values(["clinic","month"])
            .groupby("clinic", group_keys=True)
            .apply(add_metrics)
            .reset_index(level=0)
    )

latest_per_clinic = (
    df_metrics.sort_values(["clinic","month"])
              .groupby("clinic", as_index=False)
              .tail(1)
              .loc[:, ["clinic","month","cv","users","cvr","cv_ttm12","cv_3m","yoy_3m","cv_ewma"]]
)

print("✅ Metrics computed: rows=", len(df_metrics))

# 並び順（好調→不調）
# health_score = 0.6 * 最新yoy_3m + 0.4 * (最近6ヶ月のEWMA傾き / 直近12ヶ月平均CV)
def _health_scores(dfm: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for clinic, g in dfm.groupby("clinic"):
        g = g.sort_values("month")
        yoy = g["yoy_3m"].dropna().iloc[-1] if g["yoy_3m"].notna().any() else np.nan
        g2 = g.tail(6)
        if len(g2) >= 3 and g2["cv_ewma"].notna().sum() >= 3:
            x = np.arange(len(g2)); y = g2["cv_ewma"].to_numpy()
            slope = np.polyfit(x, y, 1)[0]
        else:
            slope = np.nan
        base = g.tail(12)["cv"].mean()
        slope_norm = slope / max(1.0, base) if pd.notna(slope) else np.nan
        score = (0.6*yoy if pd.notna(yoy) else 0) + (0.4*slope_norm if pd.notna(slope_norm) else 0)
        rows.append({"clinic": clinic, "score": score})
    return pd.DataFrame(rows)

scores = _health_scores(df_metrics)
present = [ja for ja in CLINIC_ALLOWED_JA if ja in df_metrics["clinic"].unique()]
order = scores.sort_values("score", ascending=False)["clinic"].tolist()
order = [c for c in order if c in present]  # 12院に限定

# 表示用に日本語の並びを出力
print("📝 並び順（好調→不調）:", order)

# === Microcharts ===
import matplotlib.pyplot as plt
def english_label(clinic: str, idx: int) -> str:
    return CLINIC_JA_TO_EN.get(clinic, f"C{idx+1}")

def small_multiples(df_metrics, clinics_order, metric="cv_ttm12", cols=12, normalize=False,
                    width=1.6, height=1.2, fontsize=7):
    n = len(clinics_order)
    cols = max(1, min(cols, n))
    rows = int(np.ceil(n / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(cols*width, rows*height), squeeze=False)
    axes_flat = axes.ravel()
    for i, clinic in enumerate(clinics_order):
        ax = axes_flat[i]
        g = df_metrics[df_metrics["clinic"] == clinic].sort_values("month")
        y = g[metric].copy()
        if normalize:
            base = y.dropna().iloc[0] if not y.dropna().empty else np.nan
            if pd.notna(base) and base != 0: y = (y / base) * 100.0
        ax.plot(g["month"], y, linewidth=1)
        if y.dropna().shape[0] > 0:
            ax.plot(g["month"].iloc[-1], y.dropna().iloc[-1], marker="o", markersize=2)
        ax.set_xticks([]); ax.set_yticks([])
        for s in ax.spines.values(): s.set_visible(False)
        ax.text(0.02, 0.80, english_label(clinic, i), transform=ax.transAxes,
                fontsize=fontsize, fontweight="bold")
    for j in range(i+1, rows*cols): fig.delaxes(axes_flat[j])
    title = f"Microcharts — {metric}" + (" (indexed=100)" if normalize else "")
    fig.suptitle(title, y=0.98, fontsize=12)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# ▼チャート前に日本語の説明を個別表示（指定文言）
print("=== 移動合計（TTM=Trailing 12-Month） ===")
print("　直近12ヶ月のCV合計。季節性を平均化（月ブレを吸収）し、中長期の実力トレンドを把握")
print("　右肩上がり＝地合い改善。横ばい＝停滞。右端の点が最新値。")
small_multiples(df_metrics, order, metric="cv_ttm12", cols=12, normalize=False)

print("=== EWMA（指数移動平均） ===")
print(f"　直近の重みを大きくする平均（半減期{HALF_LIFE_MONTHS}ヶ月）。緩やかな変化を早めに捉えるが偶発スパイクは抑制。")
print("　傾きで勢いを評価。TTMが上向き＆EWMAも上向き＝改善の確度が高い。")
small_multiples(df_metrics, order, metric="cv_ewma", cols=12, normalize=False)

print("=== 3ヶ月移動中央値（ローリングYoY） ===")
print("　3ヶ月移動合計の前年同期比。季節要因を合わせた伸び率。")
print("　0超＝前年より改善、0未満＝悪化。外れ値には注意し、持続性を重視。")
small_multiples(df_metrics, order, metric="yoy_3m", cols=12, normalize=False)

print("=== Index？ ===")
print("　各クリニックの初期値=100で正規化した月次CV。ボラティリティと最近の伸びを直感的に把握")
print("　上向き＝改善ペース加速、上下動が大きい＝月次ブレが大きい。")
small_multiples(df_metrics, order, metric="cv", cols=12, normalize=True)

# print("== Step 5/5: Writeback ==")
# if SAVE_TO_SHEET:
#     try:
#         sh_write = gc.open_by_url(WRITE_GS_URL)
#         try:
#             ws_sum = sh_write.worksheet(WRITE_SHEET_NAME)
#             ws_sum.clear()
#         except gspread.WorksheetNotFound:
#             ws_sum = sh_write.add_worksheet(title=WRITE_SHEET_NAME, rows=2000, cols=50)
#         # クリニック最新行＋health_scoreを結合して書き出し
#         out = latest_per_clinic.merge(scores, on="clinic", how="left")
#         set_with_dataframe(ws_sum, out)
#         print(f"✅ Wrote summary to {WRITE_SHEET_NAME} @ write book")
#     except Exception as e:
#         print("⚠️ Writeback failed:", e)
# else:
#     print("（書き出しは SAVE_TO_SHEET=False のためスキップ）")

# === 13ヶ月分の指標生成（EWMA傾き・health_score・アラート含む） ===
def _slope_np(arr):
    arr = np.asarray(arr, dtype=float)
    mask = np.isfinite(arr)
    if mask.sum() < 3:
        return np.nan
    x = np.arange(mask.sum())
    y = arr[mask]
    return np.polyfit(x, y, 1)[0]

df_metrics = df_metrics.sort_values(["clinic","month"]).copy()

# EWMA傾き（直近6ヶ月のスロープ；毎月計算）
df_metrics["ewma_slope"] = (
    df_metrics.groupby("clinic")["cv_ewma"]
              .apply(lambda s: s.rolling(window=6, min_periods=3).apply(_slope_np, raw=True))
              .reset_index(level=0, drop=True)
)

# 正規化用の直近12ヶ月平均CV
df_metrics["cv_mean_12"] = (
    df_metrics.groupby("clinic")["cv"]
              .transform(lambda s: s.rolling(window=12, min_periods=1).mean())
)

# 傾きの規模正規化
df_metrics["ewma_slope_norm"] = df_metrics["ewma_slope"] / df_metrics["cv_mean_12"].clip(lower=1.0)

# health_score（NaNは0で代替：無情報＝中立）
df_metrics["health_score"] = 0.6*df_metrics["yoy_3m"].fillna(0) + 0.4*df_metrics["ewma_slope_norm"].fillna(0)

# アラート条件
df_metrics["alert_yoy2"] = (
    df_metrics.groupby("clinic")["yoy_3m"]
              .transform(lambda s: s.rolling(window=2, min_periods=2)
                                   .apply(lambda a: 1.0 if np.all(a < -0.10) else 0.0, raw=True))
).fillna(0).astype(int)

df_metrics["alert_slope3"] = (
    df_metrics.groupby("clinic")["ewma_slope"]
              .transform(lambda s: s.rolling(window=3, min_periods=3)
                                   .apply(lambda a: 1.0 if np.all(a < 0) else 0.0, raw=True))
).fillna(0).astype(int)

df_metrics["alert"] = ((df_metrics["alert_yoy2"]==1) | (df_metrics["alert_slope3"]==1)).astype(int)

# 直近13ヶ月（当月は既に除外済み）
max_m = df_metrics["month"].max()
start_m = (max_m - pd.DateOffset(months=12)).replace(day=1)
df_13 = df_metrics[(df_metrics["month"] >= start_m) & (df_metrics["clinic"].isin(CLINIC_ALLOWED_JA))].copy()

# 保存用：扱いやすいワイド表
df_13["ym"] = df_13["month"].dt.strftime("%Y%m").astype(int)  # 例: 202508
_summary_out = df_13[[
    "clinic","ym","month","cv","users","cvr",
    "cv_ttm12","yoy_3m","ewma_slope","ewma_slope_norm","health_score",
    "alert","alert_yoy2","alert_slope3"
]].copy()

# === _summary へアップサート（重複キー: clinic×ym を上書き） ===
if SAVE_TO_SHEET:
    print("== Step 5/5: Writeback to _summary ==")
    try:
        sh_write = gc.open_by_url(WRITE_GS_URL)
        try:
            ws_sum = sh_write.worksheet("_summary-cv")
            df_exist = get_as_dataframe(ws_sum, evaluate_formulas=True, header=0)
            if df_exist is None or df_exist.empty:
                df_exist = pd.DataFrame(columns=_summary_out.columns)
            df_exist.columns = [str(c).strip() for c in df_exist.columns]
        except gspread.WorksheetNotFound:
            ws_sum = sh_write.add_worksheet(title="_summary", rows=5000, cols=50)
            df_exist = pd.DataFrame(columns=_summary_out.columns)

        # キー整形
        if "ym" in df_exist.columns:
            df_exist["ym"] = pd.to_numeric(df_exist["ym"], errors="coerce").astype("Int64")
        else:
            df_exist["ym"] = pd.Series(dtype="Int64")

        # 結合→重複キーを後勝ち（新データ優先）で除去
        merged = pd.concat([df_exist[_summary_out.columns], _summary_out], ignore_index=True)
        merged["_key"] = merged["clinic"].astype(str) + "_" + merged["ym"].astype(str)
        merged = merged.sort_values(["clinic","ym","month"]).drop_duplicates(subset=["_key"], keep="last").drop(columns=["_key"])

        # 並び：最新順（降順）で見やすく
        merged = merged.sort_values(["ym","clinic"], ascending=[False, True])

        # 書き戻し
        ws_sum.clear()
        set_with_dataframe(ws_sum, merged)
        print(f"✅ Wrote {_summary_out.shape[0]} rows to _summary (upserted last 13 months)")
    except Exception as e:
        print("⚠️ Writeback failed:", e)
else:
    print("（書き出しは SAVE_TO_SHEET=False のためスキップ）")

# =========================
# (FIX) _couns: 来店カウンセリング件数（薄いスキーマ & 余計な列を無視）
# =========================
if 'df_couns' in globals() and df_couns is not None:
    print("\n== _couns（来店カウンセリング）処理 ==")
    k = df_couns.copy().dropna(how="all")

    # 必須列のみを残す（D/E列など余計な列はここで無視）
    alias = {"年月":"month", "クリニック":"clinic", "来院数":"cv"}
    k = k.rename(columns={k0:v0 for k0,v0 in alias.items() if k0 in k.columns})
    keep = [c for c in ["month","clinic","cv"] if c in k.columns]
    k = k[keep].copy()

    # 必須列チェック
    req = ["month","clinic","cv"]
    miss = [c for c in req if c not in k.columns]
    if miss:
        raise SystemExit("❌ _couns 必須列が不足: " + str(miss))

    # 当月除外（不完全月）
    current_ym = pd.Timestamp.now(tz="Asia/Tokyo").strftime("%Y%m")
    k = k[k["month"].astype(str) != current_ym].copy()

    # month を堅牢に YYYYMM へパース（202301, 202301.0, 2023-01, 2023/01 等を許容）
    import re, numpy as np, pandas as pd
    def _parse_ym_series(s: pd.Series) -> pd.Series:
        ss = s.astype(str).str.strip()
        # 末尾の ".0" を除去 → "202301"
        ss = ss.str.replace(r"\.0$", "", regex=True)
        # 区切り文字や日本語表記を除去 → "2023-01"/"2023/01"/"2023年1月" 等 → "2023 01"
        ss = ss.str.replace(r"[\/\-年]", "", regex=True).str.replace("月", "", regex=True)
        # 先頭から6桁だけ抜く（YYYYMM）
        ss = ss.str.extract(r"(\d{6})", expand=False)
        return pd.to_datetime(ss, format="%Y%m", errors="coerce")

    k["month"] = _parse_ym_series(k["month"])
    if k["month"].isna().any():
        bad = k[k["month"].isna()].head(5)
        raise SystemExit(f"❌ _couns: month 変換失敗 上位（値に '202301.0' などが混在していないか確認）:\n{bad}")

    # 値を数値化
    k["cv"] = pd.to_numeric(k["cv"], errors="coerce").fillna(0)

    # 12院のみ
    k = k[k["clinic"].isin(CLINIC_ALLOWED_JA)].copy()
    if k.empty:
        raise SystemExit("⚠️ _couns: 12院フィルタ後0件。")

    # 院×月に集計（重複があれば和）
    k_m = (
        k.groupby(["clinic","month"], as_index=False)[["cv"]]
         .sum()
         .sort_values(["clinic","month"])
    ).rename(columns={"cv":"couns"})

    # 指標
    import numpy as np
    def _add_metrics_couns(g):
        g = g.sort_values("month").copy()
        g["couns_ttm12"]     = g["couns"].rolling(window=12, min_periods=1).sum()
        g["couns_3m"]        = g["couns"].rolling(window=3,  min_periods=1).sum()
        g["couns_3m_lastyr"] = g["couns_3m"].shift(12)
        g["yoy_3m"]          = np.where(g["couns_3m_lastyr"]>0,
                                        (g["couns_3m"]/g["couns_3m_lastyr"]) - 1, np.nan)
        g["couns_ewma"]      = g["couns"].ewm(halflife=HALF_LIFE_MONTHS, adjust=False).mean()
        return g

    try:
        m_c = (k_m.sort_values(["clinic","month"])
                 .groupby("clinic", group_keys=True)
                 .apply(_add_metrics_couns, include_groups=False)
                 .reset_index(level=0))
    except TypeError:
        m_c = (k_m.sort_values(["clinic","month"])
                 .groupby("clinic", group_keys=True)
                 .apply(_add_metrics_couns)
                 .reset_index(level=0))

    # EWMA傾き（6ヶ月）と正規化
    def _slope_np(arr):
        arr = np.asarray(arr, dtype=float)
        mask = np.isfinite(arr)
        if mask.sum() < 3: return np.nan
        x = np.arange(mask.sum()); y = arr[mask]
        return np.polyfit(x, y, 1)[0]

    m_c = m_c.sort_values(["clinic","month"]).copy()
    m_c["ewma_slope"] = (
        m_c.groupby("clinic")["couns_ewma"]
           .apply(lambda s: s.rolling(window=6, min_periods=3).apply(_slope_np, raw=True))
           .reset_index(level=0, drop=True)
    )
    m_c["couns_mean_12"]   = m_c.groupby("clinic")["couns"].transform(lambda s: s.rolling(12,1).mean())
    m_c["ewma_slope_norm"] = m_c["ewma_slope"] / m_c["couns_mean_12"].clip(lower=1.0)

    # health_score（Couns版）
    m_c["health_score"] = 0.6*m_c["yoy_3m"].fillna(0) + 0.4*m_c["ewma_slope_norm"].fillna(0)

    # アラート
    m_c["alert_yoy2"] = (
        m_c.groupby("clinic")["yoy_3m"]
           .transform(lambda s: s.rolling(2,2).apply(lambda a: 1.0 if np.all(a < -0.10) else 0.0, raw=True))
    ).fillna(0).astype(int)
    m_c["alert_slope3"] = (
        m_c.groupby("clinic")["ewma_slope"]
           .transform(lambda s: s.rolling(3,3).apply(lambda a: 1.0 if np.all(a < 0) else 0.0, raw=True))
    ).fillna(0).astype(int)
    m_c["alert"] = ((m_c["alert_yoy2"]==1) | (m_c["alert_slope3"]==1)).astype(int)

    # 直近13ヶ月
    max_mk = m_c["month"].max()
    start_mk = (max_mk - pd.DateOffset(months=12)).replace(day=1)
    z = m_c[(m_c["month"] >= start_mk) & (m_c["clinic"].isin(CLINIC_ALLOWED_JA))].copy()
    z["ym"] = z["month"].dt.strftime("%Y%m").astype(int)

    _summary_couns = z[[
        "clinic","ym","month",
        "couns",
        "couns_ttm12","yoy_3m","ewma_slope","ewma_slope_norm","health_score",
        "alert","alert_yoy2","alert_slope3"
    ]].copy()

    # 書き出し（upsert: clinic×ym）
    if SAVE_TO_SHEET:
        print("== Writeback to _summary_couns ==")
        try:
            sh_write = gc.open_by_url(WRITE_GS_URL)
            try:
                ws_sumk = sh_write.worksheet("_summary_couns")
                exist = get_as_dataframe(ws_sumk, evaluate_formulas=True, header=0)
                if exist is None or exist.empty:
                    exist = pd.DataFrame(columns=_summary_couns.columns)
                exist.columns = [str(c).strip() for c in exist.columns]
            except gspread.WorksheetNotFound:
                ws_sumk = sh_write.add_worksheet(title="_summary_couns", rows=6000, cols=50)
                exist = pd.DataFrame(columns=_summary_couns.columns)

            if "ym" in exist.columns:
                exist["ym"] = pd.to_numeric(exist["ym"], errors="coerce").astype("Int64")
            else:
                exist["ym"] = pd.Series(dtype="Int64")

            merged_k = pd.concat([exist[_summary_couns.columns], _summary_couns], ignore_index=True)
            merged_k["_key"] = merged_k["clinic"].astype(str) + "_" + merged_k["ym"].astype(str)
            merged_k = (merged_k
                        .sort_values(["clinic","ym","month"])
                        .drop_duplicates(subset=["_key"], keep="last")
                        .drop(columns=["_key"])
                        .sort_values(["ym","clinic"], ascending=[False, True]))

            ws_sumk.clear()
            set_with_dataframe(ws_sumk, merged_k)
            print(f"✅ Wrote {_summary_couns.shape[0]} rows to _summary_couns (upserted last 13 months)")
        except Exception as e:
            print("⚠️ Writeback(_summary_couns) failed:", e)
    else:
        print("（_summary_couns 書き出しは SAVE_TO_SHEET=False のためスキップ）")
else:
    print("\n⚠️ df_couns が無いため _couns の処理はスキップしました。")



In [ ]:
#@title CV Dashboard — Cell 3 (Channel Mix & Mix-Adjusted CVR → _channel_summary)

import pandas as pd, numpy as np
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

print("== Cell 3: Channel mix & Mix-Adjusted CVR ==")

# 前提: Cell 2 で作成済みの _df（当月除外・Direct除外・12院のみ）を使用
if "_df" not in globals() or _df is None or _df.empty:
    raise SystemExit("❌ _df がありません（Cell 2 を先に実行してください）")

# --- 主要チャネルのマッピング（必要に応じて調整） ---
CHANNEL_MAP = {
    "Paid Search": "Ad",
    "Cross-network": "Ad",
    "Map": "Map",                # Google Maps / Local / GBP 経由を想定
    "Organic Search": "Organic",
    "Referral": "Referral",
    "Display": "Display",
    "Social": "Social",
    # それ以外は Other に落とす
}

def map_channel(ch):
    return CHANNEL_MAP.get(str(ch), "Other")

df_ch = _df.copy()
df_ch["ch_group"] = df_ch["channel"].map(map_channel)

# --- 月次（院×月×チャネル）集計 ---
ch_month = (
    df_ch.groupby(["clinic","month","ch_group"], as_index=False)[["users","cv"]]
         .sum()
         .sort_values(["clinic","ch_group","month"])
)
ch_month["cvr"] = np.where(ch_month["users"]>0, ch_month["cv"]/ch_month["users"], np.nan)

# === 基準期の自動選定 & 再重み付け (Mix-Adjusted CVR) ===
# ルール：
#  1) 同月前年（YoY）が十分なデータ（users合計>=50）ならそれを使う  → baseline_type="YoY"
#  2) だめなら直近12ヶ月（m-12〜m-1）の加重平均（usersで重み）   → "Roll12"
#  3) それも無理なら m より前の全期間の加重平均                   → "AllPast"
MIN_BASE_USERS = 50   # YoY基準に採用する最低ユーザー数
ROLL_MIN_MONTHS = 6   # Roll12を採用する最低月数

def _weights_from_block(df_block):
    """usersシェアを重み（w）として算出"""
    u = df_block.groupby("ch_group", as_index=False)["users"].sum()
    total = u["users"].sum()
    if total <= 0 or np.isnan(total):
        return {}
    u["w"] = u["users"] / total
    return dict(zip(u["ch_group"], u["w"]))

def _cvr_by_block(df_block):
    """ブロック全体の実CVR（= Σcv / Σusers）"""
    s_users = df_block["users"].sum()
    s_cv = df_block["cv"].sum()
    return (s_cv / s_users) if s_users > 0 else np.nan

def _cvr_by_group(df_block):
    """チャネル別CVR（辞書）"""
    gg = df_block.groupby("ch_group", as_index=False)[["users","cv"]].sum()
    gg["cvr"] = np.where(gg["users"]>0, gg["cv"]/gg["users"], np.nan)
    return dict(zip(gg["ch_group"], gg["cvr"]))

rows = []
for clinic, dfc in ch_month.groupby("clinic"):
    dfc = dfc.sort_values("month")
    months = sorted(dfc["month"].unique())
    for m in months:
        now_block = dfc[dfc["month"]==m]
        # 1) YoY baseline
        m_yoy = (m - pd.DateOffset(years=1)).replace(day=1)
        yoy_block = dfc[dfc["month"]==m_yoy]
        base_type = None
        base_ref_start = None
        base_ref_end = None

        if not yoy_block.empty and yoy_block["users"].sum() >= MIN_BASE_USERS:
            base_block = yoy_block
            base_type = "YoY"
            base_ref_start = base_ref_end = m_yoy
        else:
            # 2) Rolling 12 months (m-12..m-1)
            start = (m - pd.DateOffset(months=12)).replace(day=1)
            roll_block = dfc[(dfc["month"]>=start) & (dfc["month"]<m)]
            months_in_roll = roll_block["month"].nunique()
            if (months_in_roll >= ROLL_MIN_MONTHS) and (roll_block["users"].sum() >= MIN_BASE_USERS):
                base_block = roll_block
                base_type = "Roll12"
                base_ref_start, base_ref_end = start, (m - pd.DateOffset(months=1)).replace(day=1)
            else:
                # 3) All past
                allpast_block = dfc[dfc["month"]<m]
                if not allpast_block.empty and allpast_block["users"].sum() >= MIN_BASE_USERS:
                    base_block = allpast_block
                    base_type = "AllPast"
                    base_ref_start, base_ref_end = allpast_block["month"].min(), allpast_block["month"].max()
                else:
                    # 基準が作れない（初期月など）：weights=現状、base_cvr=NaN
                    base_block = None
                    base_type = "None"

        # 基準重み
        if base_block is not None:
            w_base = _weights_from_block(base_block)
            base_cvr = _cvr_by_block(base_block)           # 基準期の実CVR
            cvr_base_by_group = _cvr_by_group(base_block)  # 参考（分解用）
        else:
            # 基準なし：現状usersシェアを暫定重みとしておく
            w_base = _weights_from_block(now_block)
            base_cvr = np.nan
            cvr_base_by_group = {}

        # 現在のチャネル別CVR
        cvr_now_by_group = _cvr_by_group(now_block)

        # Mix-Adjusted CVR（基準重み×現在CVR）
        # ないチャネルは0重みでOK
        mixadj = 0.0
        for g, w in w_base.items():
            cg = cvr_now_by_group.get(g, np.nan)
            mixadj += w * (0.0 if np.isnan(cg) else cg)

        # 実CVR（現在）
        actual_cvr = _cvr_by_block(now_block)

        # 寄与分解（純粋改善＝CVR自体の改善、ミックス効果）
        # base_cvr は「基準期の実CVR」。純粋改善 = mixadj - base_cvr
        pure_impr = mixadj - base_cvr if pd.notna(base_cvr) else np.nan
        mix_effect = actual_cvr - mixadj if pd.notna(mixadj) else np.nan

        # 主要チャネル（Ad/Map/Organic）の比率（基準 vs 現状）と純粋改善寄与（参考）
        key_groups = ["Ad","Map","Organic"]
        weights_base = {f"w_base_{g}": w_base.get(g, 0.0) for g in key_groups}
        weights_now  = {f"w_now_{g}": _weights_from_block(now_block).get(g, 0.0) for g in key_groups}
        pure_by_g = {}
        if base_block is not None:
            for g in key_groups:
                cvr_b = cvr_base_by_group.get(g, np.nan)
                cvr_n = cvr_now_by_group.get(g, np.nan)
                w     = w_base.get(g, 0.0)
                pure_by_g[f"pure_{g}"] = w * ((0.0 if np.isnan(cvr_n) else cvr_n) - (0.0 if np.isnan(cvr_b) else cvr_b))
        else:
            for g in key_groups:
                pure_by_g[f"pure_{g}"] = np.nan

        rows.append({
            "clinic": clinic,
            "month": m,
            "ym": int(m.strftime("%Y%m")),
            "users": now_block["users"].sum(),
            "cv": now_block["cv"].sum(),
            "actual_cvr": actual_cvr,
            "mixadj_cvr": mixadj,
            "base_cvr": base_cvr,
            "pure_impr": pure_impr,   # = mixadj_cvr - base_cvr
            "mix_effect": mix_effect, # = actual_cvr - mixadj_cvr
            "baseline_type": base_type,
            "base_start_ym": None if base_ref_start is None else int(base_ref_start.strftime("%Y%m")),
            "base_end_ym": None if base_ref_end is None else int(base_ref_end.strftime("%Y%m")),
            **weights_base, **weights_now, **pure_by_g
        })

df_mix_summary = pd.DataFrame(rows).sort_values(["clinic","month"])

print("✅ Mix-Adjusted summary built:", df_mix_summary.shape)

# --- 書き出し：_channel_summary（重複キー clinic×ym を上書き） ---
SAVE_TO_SHEET = True if 'SAVE_TO_SHEET' in globals() and SAVE_TO_SHEET else False
if SAVE_TO_SHEET:
    print("== Writeback to _channel_summary ==")
    try:
        sh_write = gc.open_by_url(WRITE_GS_URL)
        try:
            ws_ch = sh_write.worksheet("_channel_summary")
            df_exist = get_as_dataframe(ws_ch, evaluate_formulas=True, header=0)
            if df_exist is None or df_exist.empty:
                df_exist = pd.DataFrame(columns=df_mix_summary.columns)
            df_exist.columns = [str(c).strip() for c in df_exist.columns]
        except gspread.WorksheetNotFound:
            ws_ch = sh_write.add_worksheet(title="_channel_summary", rows=10000, cols=100)
            df_exist = pd.DataFrame(columns=df_mix_summary.columns)

        # upsert（clinic×ym 後勝ち）
        need_cols = list(df_mix_summary.columns)
        for c in need_cols:
            if c not in df_exist.columns: df_exist[c] = pd.Series(dtype="float64" if c not in ["clinic","baseline_type"] else "object")

        merged = pd.concat([df_exist[need_cols], df_mix_summary[need_cols]], ignore_index=True)
        merged["_key"] = merged["clinic"].astype(str) + "_" + merged["ym"].astype(str)
        merged = merged.sort_values(["clinic","ym","month"]).drop_duplicates(subset=["_key"], keep="last").drop(columns=["_key"])
        merged = merged.sort_values(["ym","clinic"], ascending=[False, True])

        ws_ch.clear()
        set_with_dataframe(ws_ch, merged)
        print(f"✅ Wrote {df_mix_summary.shape[0]} rows to _channel_summary (upsert)")
    except Exception as e:
        print("⚠️ Writeback(_channel_summary) failed:", e)
else:
    print("（書き出しは SAVE_TO_SHEET=False のためスキップ）")

print("✅ Cell 3 complete.")


In [ ]:
#@title Couns Microcharts — Cell 4 (TTM / YoY_3M / EWMA Slope / Health Score)

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# Charts in ASCII only to avoid JP font issues
matplotlib.rcParams["font.family"] = "DejaVu Sans"
matplotlib.rcParams["axes.unicode_minus"] = True

# ---- Guards ----
if 'm_c' not in globals() or m_c is None or m_c.empty:
    raise SystemExit("❌ m_c (couns metrics) is empty. Run Cell 2 first (the _couns block).")
if 'CLINIC_JA_TO_EN' not in globals():
    # fallback from CLINIC_FILTER if needed
    try:
        CLINIC_JA_TO_EN = {ja: en for ja, en in CLINIC_FILTER}
    except Exception:
        CLINIC_JA_TO_EN = {}

# ---- Sort clinics: good → poor by latest health_score ----
latest_c = (m_c.sort_values(["clinic","month"])
              .groupby("clinic", as_index=False)
              .tail(1)[["clinic","health_score"]])
order_couns = (latest_c.sort_values("health_score", ascending=False)["clinic"].tolist())

# Fallback: keep declared order if something goes wrong
if not order_couns:
    order_couns = [ja for ja, _ in CLINIC_FILTER]

def en_label(clinic_ja: str, idx: int) -> str:
    return CLINIC_JA_TO_EN.get(clinic_ja, f"C{idx+1}")

# ---- Small multiples helper ----
def small_multiples_couns(df, clinics, metric, cols=12, last_n=13,
                          ylim=None, title=""):
    """
    df: m_c (couns metrics)
    metric: one of ["couns_ttm12","yoy_3m","ewma_slope","health_score"]
    cols: number of columns in the grid (e.g., 12)
    last_n: use last N points per clinic (e.g., 13 to match Sheets)
    ylim: tuple(ymin, ymax) or None for autoscale
    """
    n = len(clinics)
    cols = max(1, min(cols, n))
    rows = int(np.ceil(n / cols))

    fig, axes = plt.subplots(rows, cols,
                             figsize=(cols*1.6, rows*1.2),
                             squeeze=False)
    axes_flat = axes.ravel()

    for i, clinic in enumerate(clinics):
        ax = axes_flat[i]
        g = df[df["clinic"] == clinic].sort_values("month").copy()
        if last_n is not None and last_n > 0:
            g = g.tail(last_n)

        y = g[metric]
        ax.plot(g["month"], y, linewidth=1)
        if y.dropna().shape[0] > 0:
            ax.plot(g["month"].iloc[-1], y.dropna().iloc[-1], marker="o", markersize=2)

        # minimal look
        ax.set_xticks([]); ax.set_yticks([])
        for s in ax.spines.values():
            s.set_visible(False)

        ax.text(0.02, 0.80, en_label(clinic, i),
                transform=ax.transAxes, fontsize=7, fontweight="bold")

        if ylim is not None:
            ax.set_ylim(ylim[0], ylim[1])

    # remove unused axes
    for j in range(i+1, rows*cols):
        fig.delaxes(axes_flat[j])

    if title:
        fig.suptitle(title, y=0.98, fontsize=12)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# ---- Explanations (JP) + Charts (EN in-figure) ----
COLS = 12        # 横並び本数（Sheetsと同じ見栄え）
LAST_N = 13      # 最新13点（Sheetsのスパークラインと一致）

print("=== 移動合計（TTM=Trailing 12-Month） ===")
print("  直近12ヶ月の来店カウンセリング合計。季節性を平均化（月ブレを吸収）し、中長期の実力トレンドを把握。")
print("  右肩上がり＝地合い改善。横ばい＝停滞。右端の点が最新値。")
small_multiples_couns(m_c, order_couns, metric="couns_ttm12",
                      cols=COLS, last_n=LAST_N, ylim=None,
                      title="Couns — TTM (Trailing 12-Month)")

print("=== EWMA（指数移動平均）の傾き ===")
print("  直近の重みを大きくする平均（半減期=Cell 2と同じ）。傾きが上向きなら勢いが出ている。偶発スパイクは抑制。")
print("  TTMが上向き＆EWMA傾きもプラスなら“改善の確度”が高い。")
small_multiples_couns(m_c, order_couns, metric="ewma_slope",
                      cols=COLS, last_n=LAST_N, ylim=None,
                      title="Couns — EWMA Slope (last 6m)")

print("=== 3ヶ月移動合計の前年同期比（ローリングYoY） ===")
print("  季節要因を合わせた伸び率。0超＝前年より改善、0未満＝悪化。外れ値より持続性を重視。")
small_multiples_couns(m_c, order_couns, metric="yoy_3m",
                      cols=COLS, last_n=LAST_N, ylim=(-1, 1),
                      title="Couns — Rolling YoY (3-month sum)")

print("=== health_score（総合スコア） ===")
print("  定義：0.6*YoY_3M + 0.4*(EWMA傾きの規模正規化)。短中期の勢いと地合いを1指標に集約。")
print("  0付近は横ばい、プラスは改善、マイナスは悪化。")
small_multiples_couns(m_c, order_couns, metric="health_score",
                      cols=COLS, last_n=LAST_N, ylim=(-1, 1),
                      title="Couns — Health Score")


In [ ]:
#@title Channel Microcharts — Cell 5 (TTM & Mix Share by Clinic)

import numpy as np, pandas as pd, matplotlib.pyplot as plt

KEY_GROUPS = ["Ad","Map","Organic"]

# 英語ラベル用マップ（CLINIC_FILTER が [(JA, EN), ...] なら自動生成）
try:
    CLINIC_JA_TO_EN = {ja: en for ja, en in CLINIC_FILTER}
except Exception:
    # 手動で指定する場合はここを編集
    CLINIC_JA_TO_EN = {
        "池袋":"Ikebukuro","博多":"Hakata","仙台":"Sendai","横浜":"Yokohama",
        "渋谷":"Shibuya","難波":"Namba","札幌":"Sapporo","天神":"Tenjin",
        "東京":"Tokyo","梅田":"Umeda","新宿":"Shinjuku"
    }

def en_label(clinic_ja: str) -> str:
    return CLINIC_JA_TO_EN.get(clinic_ja, clinic_ja)

# === ordering (same as Cell 4) ===
try:
    ORDER_CLINICS = list(order_couns)  # Cell 4 で作った最新 health_score 降順
except NameError:
    # フォールバック: CLINIC_FILTER の順 → それ以外はα順で末尾
    try:
        ORDER_CLINICS = [ja for ja, _ in CLINIC_FILTER]
    except Exception:
        ORDER_CLINICS = []

def prep_channel_ttm_and_share(ch_month: pd.DataFrame):
    z = ch_month.copy()
    z = z[z["ch_group"].isin(KEY_GROUPS)].copy()
    # TTM（12ヶ月移動合計）を clinic×ch_group ごとに
    z = z.sort_values(["clinic","ch_group","month"])
    z["cv_ttm12"] = (z.groupby(["clinic","ch_group"])["cv"]
                       .transform(lambda s: s.rolling(12, min_periods=1).sum()))
    z["users_ttm12"] = (z.groupby(["clinic","ch_group"])["users"]
                          .transform(lambda s: s.rolling(12, min_periods=1).sum()))
    # Share（users_ttm12 の比率）
    sum_users = (z.groupby(["clinic","month"])["users_ttm12"].transform("sum"))
    z["share_users_ttm12"] = np.where(sum_users>0, z["users_ttm12"]/sum_users, np.nan)
    return z

def small_multiples_by_clinic(df, metric, title, cols=12, last_n=13):
    # clinics = sorted(df["clinic"].unique().tolist(),
    #                  key=lambda c: -df[df["clinic"]==c][metric].tail(1).mean())
    uniq = df["clinic"].unique().tolist()
    if ORDER_CLINICS:
        # ORDER_CLINICS に出現するもの → 残りをα順で後ろに
        ordered = [c for c in ORDER_CLINICS if c in uniq]
        tail    = sorted([c for c in uniq if c not in ORDER_CLINICS])
        clinics = ordered + tail
    else:
        clinics = sorted(uniq)  # 最終フォールバック

    n = len(clinics); cols = min(cols, n); rows = int(np.ceil(n/cols))
    fig, axes = plt.subplots(rows, cols, figsize=(cols*1.6, rows*1.2), squeeze=False)
    axf = axes.ravel()
    for i, clinic in enumerate(clinics):
        ax = axf[i]
        w = df[df["clinic"]==clinic].copy()
        w = w.sort_values("month")
        if last_n:
            # 最新Nヶ月のみ
            last_months = w["month"].drop_duplicates().sort_values().tail(last_n)
            w = w[w["month"].isin(last_months)]
        # 各チャネルを重ね描き（色はデフォルト）
        for g in KEY_GROUPS:
            gg = w[w["ch_group"]==g]
            if gg.empty:
                continue
            ax.plot(gg["month"], gg[metric], linewidth=1, label=g)
        ax.set_xticks([]); ax.set_yticks([])
        for s in ax.spines.values(): s.set_visible(False)
        # ax.text(0.02, 0.80, clinic, transform=ax.transAxes, fontsize=7, fontweight="bold")
        ax.text(0.02, 0.80, en_label(clinic), transform=ax.transAxes, fontsize=7, fontweight="bold")
        if i==0:
            ax.legend(loc="lower right", fontsize=6, frameon=False)
    for j in range(i+1, rows*cols):
        fig.delaxes(axf[j])
    if title: fig.suptitle(title, y=0.98, fontsize=12)
    plt.tight_layout(rect=[0,0,1,0.96]); plt.show()

# ---- build & plot ----
if "ch_month" not in globals() or ch_month is None or ch_month.empty:
    raise SystemExit("❌ ch_month がありません（Cell 3 を先に実行）")

cm = prep_channel_ttm_and_share(ch_month)

print("=== Channel TTM (Ad/Map/Organic) ===")
print("  院×チャネルの12ヶ月移動合計（CV）。規模差のある院間比較は注意、院内の寄与推移を見る。")
small_multiples_by_clinic(cm, metric="cv_ttm12",
                          title="Channel CV — TTM (12M)", cols=12, last_n=13)

print("=== Channel Mix Share (Users, 12M) ===")
print("  流入構成（Users比率, 12M）。構成の変化が CV・CVR に波及。")
small_multiples_by_clinic(cm, metric="share_users_ttm12",
                          title="Channel Mix — Users Share (12M)", cols=12, last_n=13)


In [ ]:
#@title Mix-Adjusted CVR — Cell 6 (Actual vs Mix-Adjusted vs Baseline / Pure by Channel / Mix Effect)

import numpy as np, pandas as pd, matplotlib.pyplot as plt

# --- guards & helpers ---
if "df_mix_summary" not in globals() or df_mix_summary is None or df_mix_summary.empty:
    raise SystemExit("❌ df_mix_summary がありません（Cell 3 を先に実行）")

try:
    ORDER_CLINICS = list(order_couns)   # Cell 4 と同じ順
except Exception:
    try:
        ORDER_CLINICS = [ja for ja, _ in CLINIC_FILTER]
    except Exception:
        ORDER_CLINICS = sorted(df_mix_summary["clinic"].unique().tolist())

try:
    CLINIC_JA_TO_EN = {ja: en for ja, en in CLINIC_FILTER}
except Exception:
    CLINIC_JA_TO_EN = {}

def en_label(clinic_ja: str) -> str:
    return CLINIC_JA_TO_EN.get(clinic_ja, clinic_ja)

KEY_GROUPS = ["Ad","Map","Organic"]
COLS      = 12
LAST_N    = 13

def _clinic_order(seq):
    uniq = list(dict.fromkeys(seq))  # keep order
    ordered = [c for c in ORDER_CLINICS if c in uniq]
    tail = [c for c in uniq if c not in ORDER_CLINICS]
    return ordered + tail

def _grid(n, cols=12):
    cols = max(1, min(cols, n))
    rows = int(np.ceil(n/cols))
    return rows, cols

# --- 1) Actual vs Mix-Adjusted vs Baseline (CVR) ---
print("=== Actual vs Mix-Adjusted vs Baseline CVR (last 13 months) ===")
d = df_mix_summary.copy().sort_values(["clinic","month"])
clinics = _clinic_order(d["clinic"].unique())
rows, cols = _grid(len(clinics), COLS)

fig, axes = plt.subplots(rows, cols, figsize=(cols*1.6, rows*1.2), squeeze=False)
axf = axes.ravel()
for i, c in enumerate(clinics):
    ax = axf[i]
    g = d[d["clinic"]==c].sort_values("month").tail(LAST_N)
    for col in ["actual_cvr","mixadj_cvr","base_cvr"]:
        if col in g.columns:
            ax.plot(g["month"], g[col], linewidth=1)
    if len(g):
        ax.plot(g["month"].iloc[-1], g["actual_cvr"].iloc[-1], marker="o", markersize=2)
    ax.set_xticks([]); ax.set_yticks([])
    for s in ax.spines.values(): s.set_visible(False)
    ax.text(0.02, 0.80, en_label(c), transform=ax.transAxes, fontsize=7, fontweight="bold")
# legend once
axf[0].legend(["Actual","Mix-Adj","Baseline"], loc="lower right", fontsize=6, frameon=False)
for j in range(i+1, rows*cols): fig.delaxes(axf[j])
fig.suptitle("CVR — Actual vs Mix-Adjusted vs Baseline", y=0.98, fontsize=12)
plt.tight_layout(rect=[0,0,1,0.96]); plt.show()

# --- 2) Pure Improvement by Channel (baseline-weighted ΔCVR) ---
print("=== Pure Improvement by Channel (p.p., last 13 months) ===")
d2 = d.copy()
# p.p. に変換（*100）
for gk in KEY_GROUPS:
    col = f"pure_{gk}"
    if col in d2.columns:
        d2[col] = d2[col] * 100.0

clinics = _clinic_order(d2["clinic"].unique())
rows, cols = _grid(len(clinics), COLS)
fig, axes = plt.subplots(rows, cols, figsize=(cols*1.6, rows*1.2), squeeze=False); axf = axes.ravel()

# 目盛が切れないように ±max(|値|) を全院共通で取得（最低±1p.p.）
all_absmax = 1.0
for gk in KEY_GROUPS:
    col = f"pure_{gk}"
    if col in d2.columns:
        m = d2[col].abs().max()
        if pd.notna(m): all_absmax = max(all_absmax, float(m))
yl = (-all_absmax, all_absmax)

for i, c in enumerate(clinics):
    ax = axf[i]
    g = d2[d2["clinic"]==c].sort_values("month").tail(LAST_N)
    for gk in KEY_GROUPS:
        col = f"pure_{gk}"
        if col in g.columns:
            ax.plot(g["month"], g[col], linewidth=1, label=gk)
    ax.axhline(0, linewidth=0.8)
    ax.set_ylim(yl[0], yl[1])
    ax.set_xticks([]); ax.set_yticks([])
    for s in ax.spines.values(): s.set_visible(False)
    ax.text(0.02, 0.80, en_label(c), transform=ax.transAxes, fontsize=7, fontweight="bold")
# legend once
axf[0].legend(loc="lower right", fontsize=6, frameon=False, title="Pure ΔCVR (p.p.)")
for j in range(i+1, rows*cols): fig.delaxes(axf[j])
fig.suptitle("Pure Improvement by Channel — baseline-weighted ΔCVR (p.p.)", y=0.98, fontsize=12)
plt.tight_layout(rect=[0,0,1,0.96]); plt.show()

# --- 3) Mix Effect (actual − mix-adj) and Ad share shift (optional) ---
print("=== Mix Effect (actual − mix-adj) (p.p., last 13 months) ===")
d3 = d.copy()
d3["mix_effect_pp"] = d3["mix_effect"] * 100.0

clinics = _clinic_order(d3["clinic"].unique())
rows, cols = _grid(len(clinics), COLS)
fig, axes = plt.subplots(rows, cols, figsize=(cols*1.6, rows*1.2), squeeze=False); axf = axes.ravel()

absmax = max(1.0, float(d3["mix_effect_pp"].abs().max()))
yl = (-absmax, absmax)

for i, c in enumerate(clinics):
    ax = axf[i]
    g = d3[d3["clinic"]==c].sort_values("month").tail(LAST_N)
    ax.plot(g["month"], g["mix_effect_pp"], linewidth=1)
    ax.axhline(0, linewidth=0.8)
    ax.set_ylim(yl[0], yl[1])
    ax.set_xticks([]); ax.set_yticks([])
    for s in ax.spines.values(): s.set_visible(False)
    ax.text(0.02, 0.80, en_label(c), transform=ax.transAxes, fontsize=7, fontweight="bold")

for j in range(i+1, rows*cols): fig.delaxes(axf[j])
fig.suptitle("Mix Effect — (Actual CVR − Mix-Adjusted CVR) (p.p.)", y=0.98, fontsize=12)
plt.tight_layout(rect=[0,0,1,0.96]); plt.show()

print("Done: three microchart sets above (same order as Cell 4/5).")


In [ ]:
#@title GSC → Brand/Non-Brand集計 → Google Sheets書き出し（SAはuserdataから）
# 事前準備：
# 1) Search Console の対象プロパティで、サービスアカウントのメールを「ユーザーと権限」に追加（閲覧以上）
# 2) 書き出し先のスプレッドシートを、そのサービスアカウントに「編集者」で共有
# 3) Colabで一度だけ：from google.colab import userdata; userdata.set('GCP_SA_KEY', '<サービスアカウントJSON文字列>')
# 4) SITE_URL をあなたのプロパティに変更（URLプレフィックス: https://example.com/ 末尾スラッシュ必須 / ドメイン: sc-domain:example.com）

!pip -q install google-api-python-client gspread gspread-dataframe

import json, base64, re
from datetime import date, timedelta
import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.colab import userdata

# ===== 設定 =====
# Search Console プロパティ
SITE_URL = "https://tenjinkyousei.com/"  # 例: URLプレフィックス。ドメインプロパティなら "sc-domain:tenjinkyousei.com"
DAYS_PULL = 180                           # 取得期間（過去180日）
WINDOW_COMPARE = 90                       # 直近90日 vs その前90日

# 書き出し先のスプレッドシート
WRITE_GS_URL = "https://docs.google.com/spreadsheets/d/1zw8BEwj6pxe-n7PKyC80GFr0EzIHqS3XOpuBVMVeF-4/"

# Brand判定用（院名の日本語/英語、ブランド語）
CLINIC_FILTER = [
    ("池袋","Ikebukuro"),("博多","Hakata"),("仙台","Sendai"),
    ("横浜","Yokohama"),("渋谷","Shibuya"),("難波","Namba"),
    ("札幌","Sapporo"),("天神","Tenjin"),("東京","Tokyo"),
    ("梅田","Umeda"),("新宿","Shinjuku"),
]
BRAND_EXTRA = ["天神キュア矯正歯科","天神キュア","キュア矯正歯科","天神歯科矯正歯科"]

# ===== 認証（Colab userdata に保存した SA JSON を使用）=====
raw = userdata.get('GCP_SA_KEY')
if raw is None:
    raise SystemExit("❌ userdataに GCP_SA_KEY が見つかりません。最初に `userdata.set('GCP_SA_KEY', '<JSON>')` を実行してください。")

try:
    sa_info = json.loads(raw)
except Exception:
    # base64で保存していた場合のフォールバック
    try:
        sa_info = json.loads(base64.b64decode(raw).decode('utf-8'))
    except Exception as e:
        raise SystemExit(f"❌ GCP_SA_KEY のJSONデコードに失敗: {e}")

SCOPES = [
    "https://www.googleapis.com/auth/webmasters.readonly",  # Search Console 読み取り
    "https://www.googleapis.com/auth/spreadsheets",         # Sheets 書き込み
    "https://www.googleapis.com/auth/drive",                # gspread内部で利用
]
creds = service_account.Credentials.from_service_account_info(sa_info, scopes=SCOPES)

# クライアント
sc_service = build("searchconsole", "v1", credentials=creds, cache_discovery=False)
gc = gspread.authorize(creds)
sa_email = sa_info.get("client_email")
print("✅ SA loaded:", sa_email)

# ===== GSC 取得 =====
def gsc_query(site_url: str, dimensions, startDate, endDate, rowLimit=25000) -> pd.DataFrame:
    body = {"startDate": str(startDate), "endDate": str(endDate),
            "dimensions": dimensions, "rowLimit": rowLimit}
    res = sc_service.searchanalytics().query(siteUrl=site_url, body=body).execute()
    rows = res.get("rows", [])
    out = []
    for r in rows:
        rec = {k: v for k, v in zip(dimensions, r.get("keys", []))}
        rec.update({k: r[k] for k in ("clicks","impressions","ctr","position") if k in r})
        out.append(rec)
    return pd.DataFrame(out)

end = date.today()
start = end - timedelta(days=DAYS_PULL)
dims = ["query","page","device","date"]

try:
    df = gsc_query(SITE_URL, dims, start, end)
except Exception as e:
    raise SystemExit(f"❌ GSC API エラー: {e}\nSITE_URL がプロパティと一致しているか、SAのユーザー追加/スコープを確認。")

if df.empty:
    raise SystemExit("❌ GSC: データが空でした。SITE_URL/権限/期間を確認してください。")

# 型整備
df["date"] = pd.to_datetime(df["date"], errors="coerce")
if df["date"].isna().any():
    bad = df[df["date"].isna()].head()
    raise SystemExit(f"❌ GSC: dateのパース失敗。例\n{bad}")

# ctr/positionはAPIが返す値を使う（ctrは割合、positionは平均掲載順位）
df["ctr"] = pd.to_numeric(df["ctr"], errors="coerce")
df["position"] = pd.to_numeric(df["position"], errors="coerce")
df["clicks"] = pd.to_numeric(df["clicks"], errors="coerce")
df["impressions"] = pd.to_numeric(df["impressions"], errors="coerce")

df["month"] = df["date"].values.astype("datetime64[M]")

# ===== Brand/Non-Brand ラベリング =====
# names = [re.escape(ja) for ja,_ in CLINIC_FILTER] + [re.escape(en) for _,en in CLINIC_FILTER]
names = [re.escape(x) for x in BRAND_EXTRA]
BRAND_REGEX = re.compile("|".join(names), flags=re.IGNORECASE) if names else re.compile("$^")  # 何もなければ全不一致

df["brand_flag"] = df["query"].astype(str).apply(lambda q: bool(BRAND_REGEX.search(q)))
df["brand"] = np.where(df["brand_flag"], "Brand", "Non-Brand")

# ===== 月×brand×device 集計（CTR/PositionはImpr重み）=====
agg0 = (df.groupby(["month","brand","device"], as_index=False)
          .agg(clicks=("clicks","sum"), impressions=("impressions","sum")))
agg0["ctr"] = np.where(agg0["impressions"]>0, agg0["clicks"]/agg0["impressions"], np.nan)

# positionのImpr重み平均
pos_w = (df.groupby(["month","brand","device"])
           .apply(lambda g: (g["position"]*g["impressions"]).sum()/g["impressions"].sum()
                  if g["impressions"].sum()>0 else np.nan)
           .reset_index(name="position"))
agg = agg0.merge(pos_w, on=["month","brand","device"], how="left")

# ===== 直近90日 vs その前90日 比較 =====
cut = df["date"].max() - pd.Timedelta(days=WINDOW_COMPARE)
df["period"] = np.where(df["date"] > cut, "last90d", "prev90d")

comp0 = (df.groupby(["brand","device","period"], as_index=False)
           .agg(clicks=("clicks","sum"), impressions=("impressions","sum")))
comp0["ctr"] = np.where(comp0["impressions"]>0, comp0["clicks"]/comp0["impressions"], np.nan)
pos_w2 = (df.groupby(["brand","device","period"])
            .apply(lambda g: (g["position"]*g["impressions"]).sum()/g["impressions"].sum()
                   if g["impressions"].sum()>0 else np.nan)
            .reset_index(name="position"))
comp = comp0.merge(pos_w2, on=["brand","device","period"], how="left")

# オプション：見やすい差分テーブル（brand×deviceで last−prev のΔも出す）
pivot_last = comp[comp["period"]=="last90d"].set_index(["brand","device"])
pivot_prev = comp[comp["period"]=="prev90d"].set_index(["brand","device"])
delta = pivot_last[["clicks","impressions","ctr","position"]].join(
    pivot_prev[["clicks","impressions","ctr","position"]], how="outer", lsuffix="_last", rsuffix="_prev"
)
for col in ["clicks","impressions","ctr","position"]:
    delta[f"delta_{col}"] = delta[f"{col}_last"] - delta[f"{col}_prev"]
delta = delta.reset_index()

# ===== Sheets 書き出し =====
def write_ws(sh, title, data: pd.DataFrame):
    import gspread
    from gspread_dataframe import set_with_dataframe
    try:
        ws = sh.worksheet(title); ws.clear()
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=title, rows=max(1000, len(data)+10), cols=max(26, len(data.columns)+2))
    set_with_dataframe(ws, data)
    print(f"✅ Wrote {title}: {data.shape[0]} rows x {data.shape[1]} cols")

try:
    sh = gc.open_by_url(WRITE_GS_URL)
except Exception as e:
    raise SystemExit(f"❌ Sheetsを開けません。{e}\nスプレッドシートを {sa_email} に『編集者』で共有してください。")

# 並べ替えと書き出し
agg_out = agg.sort_values(["month","brand","device"]).copy()
comp_out = comp.sort_values(["brand","device","period"]).copy()
delta_out = delta.sort_values(["brand","device"]).copy()

write_ws(sh, "_gsc_month_brand", agg_out)
write_ws(sh, "_gsc_comp_brand90", comp_out)
write_ws(sh, "_gsc_comp_brand90_delta", delta_out)

print("🎉 Done.\n- _gsc_month_brand: 月×Brand/Non-Brand×Device の集計\n- _gsc_comp_brand90: 90日比較（last/prev 行）\n- _gsc_comp_brand90_delta: last−prev の差分（クリック/表示/CTR/掲載順位）")


In [ ]:
#@title GSC detail delta (page × query × device) → _gsc_detail_delta / _gsc_page_summary
import numpy as np, pandas as pd, gspread
from gspread_dataframe import set_with_dataframe

if "df" not in globals() or df.empty:
    raise SystemExit("❌ df が見つかりません（GSC取得セルを先に実行）")

# 期間フラグ（直近90日 vs その前90日）
cut = df["date"].max() - pd.Timedelta(days=WINDOW_COMPARE)
z = df.copy()
z["period"] = np.where(z["date"] > cut, "last90d", "prev90d")

# --- page×query×device×brand×period で集計 ---
keys = ["brand","device","page","query","period"]

# クリック/表示を合算
base = (z.groupby(keys, as_index=False)
          .agg(clicks=("clicks","sum"), impressions=("impressions","sum")))

# CTR は clicks / impressions（Impr重みの平均に一致）
base["ctr"] = np.where(base["impressions"]>0, base["clicks"]/base["impressions"], np.nan)

# position は Impr重み平均
wpos = (z.assign(w=lambda d: d["position"]*d["impressions"])
          .groupby(keys, as_index=False)
          .agg(w=("w","sum"), imp=("impressions","sum")))
wpos["position"] = np.where(wpos["imp"]>0, wpos["w"]/wpos["imp"], np.nan)

g = base.merge(wpos[keys+["position"]], on=keys, how="left")

# last / prev を横持ち化
last = g[g["period"]=="last90d"].drop(columns="period").add_suffix("_last")
prev = g[g["period"]=="prev90d"].drop(columns="period").add_suffix("_prev")

# キーを復元して結合
for df_ in (last, prev):
    for k in ["brand","device","page","query"]:
        df_[k] = df_[f"{k}_last"] if f"{k}_last" in df_.columns else df_[f"{k}_prev"]
detail = pd.merge(
    last.drop(columns=[c for c in last.columns if c.endswith("_last") and c.split("_last")[0] in ["brand","device","page","query"]]),
    prev.drop(columns=[c for c in prev.columns if c.endswith("_prev") and c.split("_prev")[0] in ["brand","device","page","query"]]),
    on=["brand","device","page","query"], how="outer"
)

# 欠損処理と差分
for c in ["clicks_last","impressions_last","clicks_prev","impressions_prev"]:
    detail[c] = detail[c].fillna(0)
# CTR/pos は計算不能なら NaNのまま
detail["delta_clicks"]      = detail["clicks_last"]      - detail["clicks_prev"]
detail["delta_impressions"] = detail["impressions_last"] - detail["impressions_prev"]
detail["delta_ctr"]         = (detail["ctr_last"].fillna(0) - detail["ctr_prev"].fillna(0))
detail["delta_position"]    = (detail["position_last"] - detail["position_prev"])

# ノイズ除去（対象の表示回数が極小のものを落とす）
detail = detail[(detail["impressions_last"] + detail["impressions_prev"]) >= 20].copy()

# 優先度スコア（CTR悪化×規模）※任意
detail["priority_score"] = (-detail["delta_ctr"].fillna(0)) * detail["impressions_last"]

# しきい値ベースのフラグ（初期値：±0.5位/ -0.5pp）
POS_WINDOW = 0.5   # 「順位はほぼ同じ」の範囲
CTR_DROP   = 0.005 # CTR 悪化の閾値（0.5pp）
POS_WORSE  = 0.5   # コンテンツ劣化判定の順位悪化幅

detail["title_flag"]   = ((detail["delta_position"].abs() <= POS_WINDOW) & (detail["delta_ctr"] < -CTR_DROP)).astype(int)
detail["content_flag"] = (detail["delta_position"] >= POS_WORSE).astype(int)

# 並び替え
detail_sorted = detail.sort_values(["priority_score"], ascending=False)

# --- page単位の要約（どのページを直すべきか） ---
page_sum = (detail.assign(title_cnt=detail["title_flag"], content_cnt=detail["content_flag"])
                 .groupby(["brand","device","page"], as_index=False)
                 .agg(queries=("query","nunique"),
                      title_cnt=("title_cnt","sum"),
                      content_cnt=("content_cnt","sum"),
                      impr_last=("impressions_last","sum"),
                      ctr_last=("clicks_last","sum")))
page_sum["ctr_last"] = np.where(page_sum["impr_last"]>0, page_sum["ctr_last"]/page_sum["impr_last"], np.nan)
page_sum = page_sum.sort_values(["title_cnt","content_cnt","impr_last"], ascending=[False,False,False])

# --- Sheets 書き出し ---
try:
    sh = gc.open_by_url(WRITE_GS_URL)
except Exception as e:
    raise SystemExit(f"❌ Sheetsを開けません。{e}")

def write_ws(title, data):
    try:
        ws = sh.worksheet(title); ws.clear()
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=title, rows=max(1000, len(data)+50), cols=40)
    set_with_dataframe(ws, data)
    print(f"✅ Wrote {title}: {data.shape[0]} rows")

write_ws("_gsc_detail_delta", detail_sorted[
    ["brand","device","page","query",
     "clicks_last","impressions_last","ctr_last","position_last",
     "clicks_prev","impressions_prev","ctr_prev","position_prev",
     "delta_clicks","delta_impressions","delta_ctr","delta_position",
     "priority_score","title_flag","content_flag"]
])
write_ws("_gsc_page_summary", page_sum)
print("🎉 Done: _gsc_detail_delta / _gsc_page_summary を出力しました。")
